# Select a +/- 15 days period around one date in NATL60 dataset

## Imports and magics (for jupyter)

In [1]:
## magics

%load_ext version_information 
%version_information numpy,xarray,netCDF4,dask

Software versions
Python 2.7.11 64bit [GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
IPython 4.2.0
OS Linux 3.16.0 4 amd64 x86_64 with debian 8.5
numpy 1.11.1
xarray 0.9.1
netCDF4 1.2.4
dask 0.15.2
Wed Sep 13 16:02:50 2017 CEST

In [2]:
## path for mdules

import sys
sys.path.insert(0,"/home/albert/lib/python")

In [3]:
## imports

import numpy as np
import dask
import xarray as xr
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import pandas as pd 
import datetime
import glob as glob
import matplotlib.gridspec as gridspec
from mpl_toolkits.basemap import Basemap
import time
from dask.diagnostics import ProgressBar
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

## Datasets

   - NATL60 CJM165 simulation that runs from June 15th, 2012 to September 30th, 2013, we only have local acces to July 2012 (407Go of data)

In [4]:
#NATL60

dir="/media/extra/DATA/NATL60/NATL60-CJM165-S/3d/"
files="NATL60-CJM165_y2012m07d*.1d_gridT.nc"

tfiles=dir+files

!ls $tfiles


/media/extra/DATA/NATL60/NATL60-CJM165-S/3d/NATL60-CJM165_y2012m07d01.1d_gridT.nc
/media/extra/DATA/NATL60/NATL60-CJM165-S/3d/NATL60-CJM165_y2012m07d02.1d_gridT.nc
/media/extra/DATA/NATL60/NATL60-CJM165-S/3d/NATL60-CJM165_y2012m07d03.1d_gridT.nc
/media/extra/DATA/NATL60/NATL60-CJM165-S/3d/NATL60-CJM165_y2012m07d04.1d_gridT.nc
/media/extra/DATA/NATL60/NATL60-CJM165-S/3d/NATL60-CJM165_y2012m07d05.1d_gridT.nc
/media/extra/DATA/NATL60/NATL60-CJM165-S/3d/NATL60-CJM165_y2012m07d06.1d_gridT.nc
/media/extra/DATA/NATL60/NATL60-CJM165-S/3d/NATL60-CJM165_y2012m07d07.1d_gridT.nc
/media/extra/DATA/NATL60/NATL60-CJM165-S/3d/NATL60-CJM165_y2012m07d08.1d_gridT.nc
/media/extra/DATA/NATL60/NATL60-CJM165-S/3d/NATL60-CJM165_y2012m07d09.1d_gridT.nc
/media/extra/DATA/NATL60/NATL60-CJM165-S/3d/NATL60-CJM165_y2012m07d10.1d_gridT.nc
/media/extra/DATA/NATL60/NATL60-CJM165-S/3d/NATL60-CJM165_y2012m07d11.1d_gridT.nc
/media/extra/DATA/NATL60/NATL60-CJM165-S/3d/NATL60-CJM165_y2012m07d12.1d_gridT.nc
/media/extra/DAT

## Open NATL60 files

In [36]:
##Open NATL60 files to get boundaries of domain

dsN = xr.open_mfdataset(tfiles,concat_dim='time_counter',decode_times=False, chunks={'deptht':1 ,'time_counter':1})

latN = dsN.nav_lat
lonN = dsN.nav_lon
tempN=dsN.votemper
saltN=dsN.vosaline
depN = dsN.deptht
timN = dsN.time_counter

timN.shape

(31,)

In [113]:
t = datetime.datetime(2012, 07, 15)
tsec=(t-datetime.datetime(1958,1,1,0,0)).total_seconds()
indDate=np.where((timN.values < tsec + 15*24*3600) & (timN.values > tsec - 15*24*3600))

In [114]:
tsec

1721001600.0

In [82]:
indDate[0]

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28])

In [83]:
ex = dsN['votemper'].isel(time_counter=slice(indDate[0][0],indDate[0][-1]))

In [84]:
ex.shape

(28, 300, 3454, 5422)

In [85]:
#EN4
dir_ar="/media/extra/DATA/ARGO/"
file072012="EN.4.2.0.f.profiles.g10.201207.nc"

tfile072012=dir_ar+file072012

!ls $tfile072012

/media/extra/DATA/ARGO/EN.4.2.0.f.profiles.g10.201207.nc


In [86]:
argo072012=xr.open_dataset(tfile072012)
dep_ar=argo072012['DEPH_CORRECTED']
lat_ar=argo072012['LATITUDE']
lon_ar=argo072012['LONGITUDE']
temp_ar=argo072012['POTM_CORRECTED']
salt_ar=argo072012['PSAL_CORRECTED']
day_ar=argo072012['JULD']


In [88]:
def find_date(day_ar,year,month,tday):
    prof=[]
    date=day_ar.values[:]
    for t in np.arange(0,len(date)):
        ts = pd.to_datetime(str(day_ar.values[t])) 
        dyear=ts.strftime('%Y')
        dmonth=ts.strftime('%m')
        dday=ts.strftime('%d')
        if dyear == year:
            if dmonth == month:
                if dday == tday:
                    prof.append(t)
        aprof=np.array(prof)
    return aprof

t15=find_date(day_ar,'2012','07','15')


In [100]:
truc= pd.to_datetime(str(day_ar.values[t15[0]]))

In [101]:
dyear=truc.strftime('%Y')
dmonth=truc.strftime('%m')
dday=truc.strftime('%d')
dhour=truc.strftime('%H')
dminute=truc.strftime('%M')

In [104]:
dyear

'2012'

In [102]:
print dyear,dmonth,dday,dhour,dminute

2012 07 15 23 58


In [111]:
t = datetime.datetime(int(dyear),int(dmonth),int(dday),int(dhour),int(dminute))
tsec=(t-datetime.datetime(1958,1,1,0,0)).total_seconds()
indDate=np.where((timN.values < tsec + 15*24*3600) & (timN.values > tsec - 15*24*3600))

In [112]:
tsec

1721087880.0